In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
!cp /content/drive/MyDrive/statapps-main/src/dataloader.py /content

In [10]:
import dataloader
import torchvision.models as models

In [11]:
"""
import sys
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
sys.path.insert(0,'/content/drive/My Drive/statapps-main/src/')
"""

Mounted at /content/drive/


In [ ]:
label_attribution=LabelAttribution(path_image_google='/content/drive/My Drive/ign/img', 
                                   path_mask_google='/content/drive/My Drive/ign/mask',
                                   path_metadata='/content/drive/My Drive/ign/metadata.csv',
                                   colonne_identifiant='identifiant',
                                   path_export_train_test='/content/drive/My Drive/ign',
                                   path_image_ign='/content/drive/My Drive/ign/img',
                                   path_mask_ign='/content/drive/My Drive/ign/img',
                                   use_img_google=True,
                                   use_img_ign=False
                                    )

In [ ]:
label_attribution.run()

In [ ]:
models.resnet18(pretrained=False, progress=True)